### Spacy

In [62]:
import json 

with open("de_train.json",'r')as f:
    train = json.load(f)
    
with open("de_dev.json",'r')as f:
    dev = json.load(f)
    
train[-1][1].pop(-1)
dev[-1][1].pop(-1)

''

In [2]:
import spacy
from spacy import displacy
nlp1_de = spacy.load('de_core_news_sm')
nlp2_de = spacy.load('de_core_news_md')

In [3]:
### Obtenir entités nommées annotées
import string
ponctuation = set([x for x in string.punctuation])
entity_exist = []
for x in train:
    phrase = []
    text = x[0]
    l = x[1]
    for i in l:
        a = i[0]
        b = i[1]
        while text[b]!=' ' and text[b] not in ponctuation:
            b+=1
            if b==len(text):
                break
        t = text[i[0]:b].replace('¬',"")
        phrase.append([t,i[2].upper()])
    entity_exist.append(phrase)
print(len(entity_exist),entity_exist)

105 [[['Frankreich', 'LOC']], [['Talot', 'PERS'], ['Rath', 'ORG'], ['Rarh', 'ORG'], ['Tuillerien', 'LOC'], ['Platz', 'LOC'], ['Straße', 'LOC'], ['Hause', 'LOC'], ['Straße', 'LOC'], ['Passage', 'LOC'], ['Quai', 'LOC'], ['Passage', 'LOC'], ['Seine', 'LOC'], ['Quai', 'LOC'], ['Nationalbrucke', 'LOC'], ['Platze', 'LOC'], ['Nationalbrücke', 'LOC'], ['Quai', 'LOC'], ['Platz', 'LOC'], ['Rathe', 'ORG'], ['Hof', 'LOC'], ['Passagedes', 'LOC'], ['Gebäuden', 'LOC'], ['Rath', 'ORG'], ['Nationalpallasies', 'LOC'], ['Revolutionsbrücke', 'LOC'], ['Seineufer', 'LOC'], ['Quaid', 'LOC'], ['Nationalbrücke', 'LOC'], ['Platz', 'LOC'], ['Orangerie', 'LOC'], ['Gebäude', 'LOC'], ['Versailles', 'LOC'], ['Esplanade', 'LOC'], ['Raths', 'ORG'], [' Quai', 'LOC'], [' Universitätsgasse', 'LOC'], [' Straße', 'LOC'], [' Straße', 'LOC'], [' Quai', 'LOC']], [['Frankreich', 'LOC'], ['Guillemardet', 'PERS'], ['Frankenrepublik', 'LOC'], ['Englandsarmee', 'ORG'], ['Frankreich', 'LOC'], ['Dubois', 'PERS'], ['Ludots', 'PERS'],

In [4]:
# Nettoyer les données
#
# 1. supprimer les ponctuations

import string
ponctuation = [x for x in string.punctuation]
for i in range(len(train)):
    text = train[i][0]
    for y in ponctuation:
        text = text.replace(y," ")
    text = text.replace('¬','')
    train[i][0]=text
print(train[1][0])

Gesetzgeber  Den 19  Niv   8  Jän   warddie Staatskleidung der Sekretär Redacteurs  derStaatsbothen und Thorwächter für beyde Räthebestimmt  Auf Talot s Vorschlag beschloß manüber den constitutionellen Umkreis  welchen dasgesetzgebende Corps in Zukunft inne haben soll  folgendes  Vom Tage an  da der Rath der 500 in seinen neuen Pallast installirt seyn wird  sind dieäußerlichen Bezirke für beyde Räthe folgendermassen firir  Rarh der Alten  Der Umfang des Nationalpallastes der Alten  in den Tuillerien situirt enhält gegen Westen die Straße und den Platz desCarrousel bis zum Eintritt in die Straße Nicaise am Hause Coigny vorbey bis zur Straße des Orties  die Passage de Marigny mit einbegriffen  gegen Suden den Theil des Quai der Gallerien desLouvre s von der Passage de Marigny am rechtenUfer der Seine hinab bis zum Quai der Tuilerien vom rechten Winkel des Parapets bey der Nationalbrucke bis an den Eingang zum Platze de la Coneorde  die Nationalbrücke bis zum Quai Voltairemit einbegriffen

In [57]:
from spacy import displacy
text = train[1][0]
text = text.replace("¬","")
doc1 = nlp1_de(text)
displacy.serve(doc1,style="ent")

/Users/zijian/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [5]:
def show_entity_model1(exemple,i):
    text = exemple[0]
    result = exemple[1]
    entite = nlp1_de(text)
    res = [[ent.text.strip(),ent.start_char,ent.end_char,ent.label_] for ent in entite.ents]
    statistique = stats(res,entity_exist[i])
    return statistique

def stats(res,entity):
    trouvé_correct,incorrect,non= [],[],[]
    entity_list = entity[:]
    entity_set = set([x[0] for x in entity])
    candidat_set = set()
    #print(entity_list," ",entity_set," ",candidat_set)
    for x in res:
        l = x[0].split(" ")
        for y in l:
            candidat_set.add(y)
    #print(candidat_set)
    for x in res:
        l = x[0].split(" ")
        flag = False
        for y in l:
            if y in entity_set:
                trouvé_correct.append([x[0],x[3]])
                flag = True
                break
        if not flag:
            incorrect.append([x[0],x[3]])
    for i in range(len(entity_list)):
        if entity_list[i][0] not in candidat_set:
            non.append(entity_list[i])
    return trouvé_correct,incorrect,non
    

trouvée = []
incorrect = []
non = []
for i in range(len(train)):
    res = show_entity_model1(train[i],i)
    trouvée+=res[0]
    incorrect+=res[1]
    non+=res[2]
    
print(len(trouvée),len(incorrect),len(non))

1961 2435 1202


In [6]:
def show_entity_model2(exemple,i):
    text = exemple[0]
    result = exemple[1]
    entite = nlp2_de(text)
    res = [[ent.text.strip(),ent.start_char,ent.end_char,ent.label_] for ent in entite.ents]
    statistique = stats(res,entity_exist[i])
    return statistique

t = []
inc = []
n = []
for i in range(len(train)):
    res = show_entity_model2(train[i],i)
    t+=res[0]
    inc+=res[1]
    n+=res[2]
    
print(len(t),len(inc),len(n))

2123 3165 1100


In [8]:
print(len(non))
i = 0
while i<len(non):
    if len(non[i][0])<=2:
        non.remove(non[i])
    else:
        i+=1
print(len(non))
print()
print(len(n))
j = 0
while j<len(n):
    if len(n[j][0])<=2:
        n.remove(n[j])
    else:
        j+=1
print(len(n))

1202
1132

1100
1032


### Résultat Spacy

In [9]:
TP = len(trouvée)
TF = len(incorrect)
FN = len(non)
précison = 100*TP/(TP+TF)
rappel = 100*TP/(TP+FN)
F_mesure = (2*précison*rappel)/(précison+rappel)
TP1 = len(t)
TF1 = len(inc)
FN1 = len(n)
précison1 = 100*TP1/(TP1+TF1)
rappel1 = 100*TP1/(TP1+FN1)
F_mesure1 = (2*précison1*rappel1)/(précison1+rappel1)
import pandas as pd
tab = pd.DataFrame({"Precision":[précison,précison1],"Recall":[rappel,rappel1],"F-Mesure":[F_mesure,F_mesure1]},index = ["model1","model2"])
tab

,Precision,Recall,F-Mesure
model1,45.785664,63.401229,53.172451
model2,41.127470,67.290016,51.052062


In [84]:
with open("résultat/spacy model trouvé",'w',encoding="utf-8")as f:
    for x in trouvée+t:
        f.write(x[0])
        f.write(" ")
        f.write(x[1])
        f.write("\n")
with open("résultat/spacy model non trouvé",'w',encoding="utf-8")as f:
    for x in non+n:
        f.write(x[0])
        f.write(" ")
        f.write(x[1])
        f.write("\n")

In [39]:
doc2 = nlp2_de(text)
displacy.serve(doc2,style="ent")

/Users/zijian/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)


KeyboardInterrupt: 